In [1]:
from tqdm.auto import tqdm
import numpy as np
from icecream import ic

In [ ]:
PUZZLE_DIM = 4
ENCODING = "HOLE" # the hole is the one that moves
ACTIONS_ENCODING = ["U", "D", "L", "R"] # available movements
RANDOMIZE_STEPS = 100_000

In [ ]:
#state = rng.permutation(np.array([i for i in range(1, PUZZLE_DIM**2)]+[0])).reshape(PUZZLE_DIM, PUZZLE_DIM)
#state

array([[14,  2, 11,  1],
       [ 3,  8, 10,  9],
       [ 0,  7,  5,  4],
       [13,  6, 12, 15]])

In [5]:
def available_actions(state: np.ndarray) -> list[str]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append("U") 
    if x < PUZZLE_DIM - 1:
        actions.append("D")
    if y > 0:
        actions.append("L")
    if y < PUZZLE_DIM - 1:
        actions.append("R")
    return actions

def swap(state, pos1, pos2):
    state[pos1], state[pos2] = state[pos2], state[pos1]

def hprintmat(mtx: np.ndarray):
    #parg = "\033[1;31m"+msg+"\033[0m"
    print(str(mtx))

def do_action(state: np.ndarray, action: str,*,print=False) -> np.ndarray:
    new_state = state.copy()
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    if action == "U":
        swap(new_state, (x, y), (x-1, y))
    elif action == "D":
        swap(new_state, (x, y), (x+1, y))
    elif action == "L":
        swap(new_state, (x, y), (x, y-1))
    elif action == "R":
        swap(new_state, (x, y), (x, y+1))
    else:
        raise ValueError("Invalid action")
    return new_state

In [ ]:
state
rng = np.random.Generator(np.random.PCG64([PUZZLE_DIM, RANDOMIZE_STEPS]))
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, rng.choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[ 0, 14,  9,  6],
       [ 1, 11, 10,  2],
       [ 4,  8,  7, 15],
       [ 5, 12,  3, 13]])

Evaluation: let's evaluate how does the algorithms work

In [12]:
actions = available_actions(state)
hprintmat(state)
ic(actions)
for action in actions:
    new_state = do_action(state, action,print=True)

ic| actions: ['U', 'D', 'R']


[[14  2 11  1]
 [ 3  8 10  9]
 [ 0  7  5  4]
 [13  6 12 15]]


## BFS Algorithm

Let's try to implement a simple BFS algorithm using a simple 

In [ ]:
from collections import deque

# Create an empty deque
frontier = deque()
visited = set()

# Add the initial state to the frontier
frontier.append(state)


In [ ]:
cycle=0
while frontier:
    current_state = frontier.popleft()
    visited.add(tuple(current_state.flatten())) # avoid using np.array as keys in a set
    if np.array_equal(current_state, np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12], [13, 14, 15, 0]])):
        print("Solved!")
        break
    actions = available_actions(current_state)
    for action in actions:
        new_state = do_action(current_state, action)
        if tuple(new_state.flatten()) not in visited:
            frontier.append(new_state)
    if cycle % 1000 == 0:
        print(current_state)
